# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [ ]:
!pip install pyconll
!pip install spacy_udpipe

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
#!wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://drive.google.com/file/d/1fsbrTM3UUzTy3Hz4KRcLXQRLDuJWXLnr/view?usp=sharing

# Файл ru_syntagrus-ud-train.conllu скопировал вручную с компа в файл в нотбук
!wget -O .datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu

--2023-08-18 19:43:15--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14970950 (14M) [text/plain]
Saving to: ‘./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu’

./stepik-dl-nlp/dat 100%[===================>]  14.28M  --.-KB/s    in 0.07s   

2023-08-18 19:43:16 (207 MB/s) - ‘./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu’ saved [14970950/14970950]



In [ ]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu

--2023-08-18 19:43:17--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu’

./stepik-dl-nlp/dat 100%[===================>]  38.85M  --.-KB/s    in 0.1s    

2023-08-18 19:43:19 (354 MB/s) - ‘./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu’ saved [40736581/40736581]

--2023-08-18 19:43:19--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubuser

In [ ]:
full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [ ]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent if token.form)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [ ]:
all_train_texts = [' '.join(token.form for token in sent if token.form) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [ ]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 142
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [ ]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [ ]:
import torch
from torch.utils.data import TensorDataset

from dlnlputils.pipeline import predict_with_model
#from .base import tokenize_corpus


def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
    inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long)
    targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

    for sent_i, sent in enumerate(sentences):
        for token_i, token in enumerate(sent):
            targets[sent_i, token_i] = label2id.get(token.upos, 0) # Если нет ключа token.upos в label2id то .get вернёт 0

            if token.form:
              for char_i, char in enumerate(token.form):
                inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)

    return inputs, targets

In [ ]:
train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

In [ ]:
train_inputs[1][:5] # Первые 5 слов предложения №2 (т.к индекс 1)

tensor([[ 0, 38,  4, 25,  4, 11, 19,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 17, 16, 10,  4, 12, 11,  3,  7,  6, 20,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9, 12, 20, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 15,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [ ]:
train_labels[1] # Для первого слова (train_inputs[1][0]) в предложении 2 мы должны предсказать класс 8

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

## Вспомогательная свёрточная архитектура

nn.ModuleList — это просто список Python (хотя он полезен, поскольку параметры можно обнаружить и обучить с помощью оптимизатора). В то время как nn.Sequential — это модуль, который последовательно запускает компонент на входе.

In [ ]:
# Реализация аналога простого ResNet
# kernel_size=3 означает что в одном токене (слове) мы будем "прощупывать" (учитывать влияние контекста) 3-х рядом стоящих символов

class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2), # Сверточный слой принимает и возвращает одно и тоже число каналов; padding для неизменности размера тензора
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)   # Skip Connections
        return x

## Предсказание частей речи на уровне отдельных токенов

In [ ]:
e = torch.tensor([[[1],[2]],[[3],[4]]])
print(f'Старый вид e:')
print(e)
print(f'Старый размер: {e.shape}')
batch_size_n, max_sent_len_n, max_token_len_n =  e.shape
print(f'batch_size_n = {batch_size_n}, max_sent_len_n = {max_sent_len_n}')
e = e.view(batch_size_n * max_sent_len_n, max_token_len_n)
print(f'Новый вид e:')
print(e)
print(f'Новый размер e')
print(e.shape)

Старый вид e:
tensor([[[1],
         [2]],

        [[3],
         [4]]])
Старый размер: torch.Size([2, 2, 1])
batch_size_n = 2, max_sent_len_n = 2
Новый вид e:
tensor([[1],
        [2],
        [3],
        [4]])
Новый размер e
torch.Size([4, 1])


In [ ]:
vocab_size_n = 10 # 10 тензоров
embedding_size_n = 3
ee = nn.Embedding(vocab_size_n, embedding_size_n, padding_idx=0)(e) # Создать 10 тензоров эмбедингов и выбрать из них тензора с номерами из списка "e"
ee

tensor([[[-0.4339,  0.8487,  0.6920]],

        [[-0.3160, -2.1152,  0.3223]],

        [[-1.2633,  0.3500,  0.3081]],

        [[ 0.1198,  1.2377, -0.1435]]], grad_fn=<EmbeddingBackward0>)

In [ ]:
ee = ee.permute(0, 2, 1)
ee

tensor([[[-0.4339],
         [ 0.8487],
         [ 0.6920]],

        [[-0.3160],
         [-2.1152],
         [ 0.3223]],

        [[-1.2633],
         [ 0.3500],
         [ 0.3081]],

        [[ 0.1198],
         [ 1.2377],
         [-0.1435]]], grad_fn=<PermuteBackward0>)

In [ ]:
ee.shape

torch.Size([4, 3, 1])

In [ ]:
e_features = StackedConv1d(embedding_size_n)(ee)
e_features

tensor([[[-0.0304],
         [ 0.8485],
         [ 1.0954]],

        [[-0.3182],
         [-1.5442],
         [ 0.3206]],

        [[-1.1456],
         [ 0.4405],
         [ 0.6901]],

        [[ 0.6768],
         [ 1.3504],
         [ 0.4191]]], grad_fn=<AddBackward0>)

In [ ]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num) # labels_num --- количество меток частей речи
        self.labels_num = labels_num

    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len) # Забываем что токены были объеденены в предложения

        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize Для каждого символа получить вектора
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen (Размер батча=BatchSize*MaxSentenceLen, EmbSize-количество признаков для каждого элемента ) Транспонируем тензор чтобы его подать в свёрточную нейросеть

        features = self.backbone(char_embeddings) # Содержит вектора символов уже с учетом контекста
        # Теги нужно предсказывать не для каждого символа, а для каждого токена (слова) => нужно агрегировать признаки символов чтобы получить вектор токена

        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize Получаем один вектор, количество элементов в котором соответствует EmbSize, элемент = max(столбец фиксированного инд эмбединга для всех символов рассматриваемого слова)
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum Вспоминаем что у нас есть предложения
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num) # labels_num --- количество меток частей речи
        self.labels_num = labels_num

    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len) # Забываем что токены были объеденены в предложения

        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize Для каждого символа получить вектора
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen (Размер батча=BatchSize*MaxSentenceLen, EmbSize-количество признаков для каждого элемента ) Транспонируем тензор чтобы его подать в свёрточную нейросеть

        features = self.backbone(char_embeddings) # Содержит вектора символов уже с учетом контекста
        # Теги нужно предсказывать не для каждого символа, а для каждого токена (слова) => нужно агрегировать признаки символов чтобы получить вектор токена

        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize Получаем один вектор, количество элементов в котором соответствует EmbSize, элемент = max(столбец фиксированного инд эмбединга для всех символов рассматриваемого слова)
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum Вспоминаем что у нас есть предложения
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num) # labels_num --- количество меток частей речи
        self.labels_num = labels_num

    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len) # Забываем что токены были объеденены в предложения

        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize Для каждого символа получить вектора
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen (Размер батча=BatchSize*MaxSentenceLen, EmbSize-количество признаков для каждого элемента ) Транспонируем тензор чтобы его подать в свёрточную нейросеть

        features = self.backbone(char_embeddings) # Содержит вектора символов уже с учетом контекста
        # Теги нужно предсказывать не для каждого символа, а для каждого токена (слова) => нужно агрегировать признаки символов чтобы получить вектор токена

        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize Получаем один вектор, количество элементов в котором соответствует EmbSize, элемент = max(столбец фиксированного инд эмбединга для всех символов рассматриваемого слова)
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum Вспоминаем что у нас есть предложения
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47314


In [ ]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Эпоха: 384 итераций, 40.67 сек
Среднее значение функции потерь на обучении 0.0962288748996798
Среднее значение функции потерь на валидации 0.036881465225083995
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 33.46 сек
Среднее значение функции потерь на обучении 0.03174445396871306
Среднее значение функции потерь на валидации 0.029106813344624963
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 34.30 сек
Среднее значение функции потерь на обучении 0.026835750628379174
Среднее значение функции потерь на валидации 0.027410112653333363
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 33.95 сек
Среднее значение функции потерь на обучении 0.024526089716043014
Среднее значение функции потерь на валидации 0.02460995739750048
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 34.09 сек
Среднее значение функции потерь на обучении 0.023234407031850424
Среднее значение функции потерь на валидации 0.024650209122291294

Эпоха 5
Эпоха: 384 итераций, 34.15 сек
Среднее значение функции п

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_single_token_model.state_dict(), './stepik-dl-nlp/models/single_token_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
single_token_model.load_state_dict(torch.load('./stepik-dl-nlp/models/single_token_pos.pth'))

<All keys matched successfully>

In [ ]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:12, 63.38it/s]                             


Среднее значение функции потерь на обучении 0.017917033284902573
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.88      0.94      0.91     43357
         ADP       1.00      0.99      0.99     39344
         ADV       0.89      0.88      0.89     22733
         AUX       0.86      0.72      0.79      3537
       CCONJ       0.87      0.99      0.93     15168
         DET       0.86      0.80      0.83     10781
        INTJ       0.88      0.28      0.42        50
        NOUN       0.97      0.94      0.96    103538
         NUM       0.92      0.93      0.92      5640
        PART       0.98      0.75      0.85     13556
        PRON       0.89      0.86      0.88     18733
       PROPN       0.87      0.91      0.89     14855
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.79      0.89      0.83      8057
         SYM       1.00      0.99      0.99       420
        VERB    

100%|██████████| 275/275.0 [00:04<00:00, 64.07it/s]


Среднее значение функции потерь на валидации 0.024432189762592316
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1549482
         ADJ       0.83      0.93      0.88     14471
         ADP       0.99      0.99      0.99     15062
         ADV       0.84      0.87      0.86      8085
         AUX       0.90      0.70      0.79      1518
       CCONJ       0.88      0.99      0.94      5736
         DET       0.85      0.77      0.81      4094
        INTJ       0.83      0.22      0.34        23
        NOUN       0.96      0.92      0.94     36568
         NUM       0.91      0.90      0.91      2528
        PART       0.98      0.73      0.84      4921
        PRON       0.91      0.88      0.89      8015
       PROPN       0.85      0.85      0.85      5883
       PUNCT       1.00      1.00      1.00     29463
       SCONJ       0.78      0.89      0.83      2992
         SYM       1.00      1.00      1.00       165
        VERB   

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [ ]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num

    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)

        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)

        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84370


In [ ]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=10,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

Эпоха 0
Эпоха: 384 итераций, 171.54 сек
Среднее значение функции потерь на обучении 0.08028637797300082
Среднее значение функции потерь на валидации 0.0306947062363719
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 171.38 сек
Среднее значение функции потерь на обучении 0.027914599670718115
Среднее значение функции потерь на валидации 0.020162754793568414
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 171.36 сек
Среднее значение функции потерь на обучении 0.02283729436021531
Среднее значение функции потерь на валидации 0.018671032742108448
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 171.30 сек
Среднее значение функции потерь на обучении 0.020233459166774992
Среднее значение функции потерь на валидации 0.01696559874857269
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 171.23 сек
Среднее значение функции потерь на обучении 0.018788335835173104
Среднее значение функции потерь на валидации 0.0164705785268014
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 171.34 сек
Ср

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), './stepik-dl-nlp/models/sentence_level_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
sentence_level_model.load_state_dict(torch.load('./stepik-dl-nlp/models/sentence_level_pos.pth'))

<All keys matched successfully>

In [ ]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:11, 63.98it/s]                             


Среднее значение функции потерь на обучении 0.010789771564304829
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.91      0.95      0.93     43357
         ADP       1.00      0.99      0.99     39344
         ADV       0.93      0.90      0.92     22733
         AUX       0.91      0.87      0.89      3537
       CCONJ       0.94      0.97      0.96     15168
         DET       0.92      0.91      0.92     10781
        INTJ       0.95      0.38      0.54        50
        NOUN       0.98      0.96      0.97    103538
         NUM       0.93      0.94      0.94      5640
        PART       0.96      0.90      0.92     13556
        PRON       0.95      0.91      0.93     18733
       PROPN       0.95      0.97      0.96     14855
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.85      0.95      0.90      8057
         SYM       1.00      0.99      0.99       420
        VERB    

100%|██████████| 275/275.0 [00:04<00:00, 62.87it/s]


Среднее значение функции потерь на валидации 0.015891045331954956
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1549482
         ADJ       0.87      0.94      0.90     14471
         ADP       0.99      0.99      0.99     15062
         ADV       0.90      0.89      0.89      8085
         AUX       0.93      0.86      0.90      1518
       CCONJ       0.95      0.96      0.96      5736
         DET       0.92      0.86      0.89      4094
        INTJ       1.00      0.26      0.41        23
        NOUN       0.97      0.95      0.96     36568
         NUM       0.93      0.92      0.93      2528
        PART       0.94      0.86      0.90      4921
        PRON       0.95      0.92      0.93      8015
       PROPN       0.92      0.95      0.93      5883
       PUNCT       1.00      1.00      1.00     29463
       SCONJ       0.84      0.95      0.89      2992
         SYM       1.00      1.00      1.00       165
        VERB   

## Применение полученных теггеров и сравнение

In [ ]:
single_token_pos_tagger = POSTagger(single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [ ]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [ ]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 84.58it/s]                     

мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-NOUN

глокая-ADJ куздра-NOUN штеко-ADJ будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



In [ ]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 127.44it/s]                    

мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-NOUN

глокая-ADJ куздра-NOUN штеко-NOUN будланула-NOUN бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-NOUN дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-VERB

сорок-NOUN сорок-NOUN



## Свёрточный модуль своими руками

In [ ]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)

    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [ ]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 84370


In [ ]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

Эпоха 0
Эпоха: 384 итераций, 64.86 сек
Среднее значение функции потерь на обучении 0.09295775835926179
Среднее значение функции потерь на валидации 0.02613215414014193
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 64.86 сек
Среднее значение функции потерь на обучении 0.02407210282156787
Среднее значение функции потерь на валидации 0.018735020052604745
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 64.89 сек
Среднее значение функции потерь на обучении 0.0203012618876528
Среднее значение функции потерь на валидации 0.01792207546532154
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 64.93 сек
Среднее значение функции потерь на обучении 0.01852577231572165
Среднее значение функции потерь на валидации 0.01673353584667686
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 64.99 сек
Среднее значение функции потерь на обучении 0.017565954628177376
Среднее значение функции потерь на валидации 0.01547727872165713
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 65.05 сек
Среднее зн

In [ ]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:19, 38.73it/s]                             


Среднее значение функции потерь на обучении 0.010462422855198383
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.96      0.92      0.94     43357
         ADP       1.00      0.99      0.99     39344
         ADV       0.85      0.96      0.90     22733
         AUX       0.89      0.90      0.90      3537
       CCONJ       0.91      0.99      0.95     15168
         DET       0.90      0.93      0.91     10781
        INTJ       0.93      0.26      0.41        50
        NOUN       0.97      0.97      0.97    103538
         NUM       0.95      0.95      0.95      5640
        PART       0.98      0.84      0.91     13556
        PRON       0.96      0.91      0.93     18733
       PROPN       0.96      0.95      0.95     14855
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.88      0.80      0.84      8057
         SYM       1.00      0.99      0.99       420
        VERB    

100%|██████████| 275/275.0 [00:07<00:00, 38.76it/s]


Среднее значение функции потерь на валидации 0.015999944880604744
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1549482
         ADJ       0.92      0.90      0.91     14471
         ADP       1.00      0.99      0.99     15062
         ADV       0.81      0.95      0.87      8085
         AUX       0.92      0.91      0.91      1518
       CCONJ       0.92      0.99      0.96      5736
         DET       0.89      0.88      0.89      4094
        INTJ       1.00      0.26      0.41        23
        NOUN       0.96      0.96      0.96     36568
         NUM       0.93      0.92      0.93      2528
        PART       0.98      0.81      0.89      4921
        PRON       0.96      0.92      0.94      8015
       PROPN       0.95      0.91      0.93      5883
       PUNCT       1.00      1.00      1.00     29463
       SCONJ       0.87      0.79      0.83      2992
         SYM       0.98      1.00      0.99       165
        VERB   